In [49]:
import pandas as pd 
import numpy as np
import torch
from IMUNet import *
from abc import ABC, abstractmethod
import seaborn as sns

In [27]:
datasets_path = '/home/fabian/Documents/Arbeit/olive/IMUNet/Datasets/' 
file_path = datasets_path + 'olive/lio_interpolated.csv'
dataframe = pd.read_csv(file_path)

In [63]:
test_dataframe = dataframe[['imu_1_angular_velocity_x', 'imu_1_angular_velocity_y', 'imu_1_angular_velocity_z', 'imu_1_linear_acceleration_x', 'imu_1_linear_acceleration_y', 'imu_1_linear_acceleration_z', 'position_x', 'position_y', 'position_z']]
test_dataframe
dataframe

,timestamp,imu_1_orientation_x,imu_1_orientation_y,imu_1_orientation_z,imu_1_orientation_w,imu_1_angular_velocity_x,imu_1_angular_velocity_y,imu_1_angular_velocity_z,imu_1_linear_acceleration_x,imu_1_linear_acceleration_y,...,imu_3_linear_acceleration_x,imu_3_linear_acceleration_y,imu_3_linear_acceleration_z,position_x,position_y,position_z,orientation_x,orientation_y,orientation_z,orientation_w
0,1726650717841714596,0.001356,0.002824,-0.026295,0.999619,-0.001569,-0.003268,0.000434,-0.272110,0.918579,...,-0.022939,0.871253,9.847967,-0.004760,0.047556,0.013487,0.999999,-0.000299,-0.000902,-0.000632
1,1726650717842714596,0.010195,-0.011345,0.355104,0.934701,-0.001106,-0.001667,0.000389,-0.238239,0.285303,...,-0.272110,0.918579,9.656079,-0.004760,0.047556,0.013487,0.999999,-0.000299,-0.000902,-0.000632
2,1726650717843714596,0.010527,-0.011956,0.355080,0.934699,-0.001038,-0.002177,0.000478,-0.191854,0.230596,...,-0.272110,0.918579,9.656079,-0.004760,0.047556,0.013487,0.999999,-0.000299,-0.000902,-0.000632
3,1726650717844714596,0.010527,-0.011956,0.355080,0.934699,-0.001038,-0.002177,0.000478,-0.191854,0.230596,...,-0.272110,0.918579,9.656079,-0.004760,0.047556,0.013487,0.999999,-0.000299,-0.000902,-0.000632
4,1726650717845714596,0.010527,-0.011956,0.355080,0.934699,-0.001038,-0.002177,0.000478,-0.191854,0.230596,...,-0.272110,0.918579,9.656079,-0.004921,0.051102,0.014464,0.999999,-0.000300,-0.000894,-0.000640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51813,1726650769654714596,0.010741,-0.017477,0.526063,0.850194,-0.000337,-0.001780,0.000347,-0.169390,0.409995,...,-0.212720,0.968679,9.664181,-0.450594,-0.416355,-0.003566,0.990575,0.136919,0.000466,-0.003673
51814,1726650769655714596,0.010741,-0.017477,0.526063,0.850194,-0.000337,-0.001780,0.000347,-0.169390,0.409995,...,-0.212720,0.968679,9.664181,-0.450682,-0.416294,-0.003363,0.990575,0.136920,0.000464,-0.003673
51815,1726650769656714596,0.010665,-0.017531,0.526064,0.850196,-0.000293,-0.001674,0.000578,-0.202327,0.372444,...,-0.231530,0.958780,9.676258,-0.450682,-0.416294,-0.003363,0.990575,0.136920,0.000464,-0.003673
51816,1726650769657714596,0.010665,-0.017531,0.526064,0.850196,-0.000293,-0.001674,0.000578,-0.202327,0.372444,...,-0.231530,0.958780,9.676258,-0.450759,-0.416241,-0.003185,0.990575,0.136920,0.000463,-0.003673


In [43]:
import os
import pandas as pd
import numpy as np
from torch.utils.data import Dataset

class CustomIMUDataset(Dataset):
    def __init__(self, csv_file, sequence_length=200, transform=None, target_transform=None):
        """
        Args:
            csv_file (string): Path to the CSV file with data.
            sequence_length (int): Length of the sequence window.
            transform (callable, optional): Optional transform to be applied on the features.
            target_transform (callable, optional): Optional transform to be applied on the targets.
        """
        self.data = pd.read_csv(csv_file)
        self.sequence_length = sequence_length
        self.transform = transform
        self.target_transform = target_transform

        # Preprocess the data to extract features and targets
        self.features, self.targets = self.preprocess_data()

    def preprocess_data(self):
        # Extract features and targets from the dataframe

        # For example, let's use imu_1 data as features
        features = self.data[[
            'imu_1_angular_velocity_x',
            'imu_1_angular_velocity_y',
            'imu_1_angular_velocity_z',
            'imu_1_linear_acceleration_x',
            'imu_1_linear_acceleration_y',
            'imu_1_linear_acceleration_z'
        ]].values

        # Extract targets, for example positions
        targets = self.data[[
            'position_x',
            'position_y',
            'position_z'
        ]].values

        # Convert data to float32
        features = features.astype('float32')
        targets = targets.astype('float32')

        return features, targets

    def __len__(self):
        # Number of sequences that can be formed from the data
        return len(self.features) - self.sequence_length + 1

    def __getitem__(self, idx):
        # Get a sequence of features and the corresponding target
        features_seq = self.features[idx : idx + self.sequence_length]
        target = self.targets[idx + self.sequence_length - 1]

        # if self.transform:
        #     features_seq = self.transform(features_seq)
        # if self.target_transform:
        #     target = self.target_transform(target)

        # # Transpose features to match expected input shape: [num_features, sequence_length]
        # features_seq = features_seq.T  # Shape: [num_features, sequence_length]

        return features_seq, target

In [52]:
training_data = dataset

In [66]:
# def train(batchsize=32, lr= 0.001):
lr = 0.001
batchsize = 32
# Loading data
start_t = time.time()
train_dataset = get_dataset_from_list(args.root_dir, args.train_list, args, mode='train')
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

end_t = time.time()
print('Training set loaded. Feature size: {}, target size: {}. Time usage: {:.3f}s'.format(
    train_dataset.feature_dim, train_dataset.target_dim, end_t - start_t))

val_dataset, val_loader = None, None

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

network = IMUNet(num_classes= 2, input_size= (6, 200) ,sampling_rate= 200, num_T = 32 , num_S = 64 , hidden = 64, dropout_rate = 0.5).to(device)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(network.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=10, verbose=True, eps=1e-12)
# torch.autograd.set_detect_anomaly(True)
start_epoch = 0
if args.continue_from is not None and osp.exists(args.continue_from):
    checkpoints = torch.load(args.continue_from)
    start_epoch = checkpoints.get('epoch', 0)
    network.load_state_dict(checkpoints.get('model_state_dict'))
    optimizer.load_state_dict(checkpoints.get('optimizer_state_dict'))

if args.out_dir is not None and osp.exists(osp.join(args.out_dir, 'logs')):
    summary_writer = SummaryWriter(osp.join(args.out_dir, 'logs'))
    # summary_writer.add_text('info', 'total_param: {}'.format(total_params))

step = 0
best_val_loss = np.inf

print('Start from epoch {}'.format(start_epoch))
total_epoch = start_epoch
train_losses_all, val_losses_all = [], []

# Get the initial loss.
init_train_targ, init_train_pred = run_test(network, train_loader, device, eval_mode=False)

init_train_loss = np.mean((init_train_targ - init_train_pred) ** 2, axis=0)
train_losses_all.append(np.mean(init_train_loss))
print('-------------------------')
print('Init: average loss: {}/{:.6f}'.format(init_train_loss, train_losses_all[-1]))
if summary_writer is not None:
    add_summary(summary_writer, init_train_loss, 0, 'train')

if val_loader is not None:
    init_val_targ, init_val_pred = run_test(network, val_loader, device)
    init_val_loss = np.mean((init_val_targ - init_val_pred) ** 2, axis=0)
    val_losses_all.append(np.mean(init_val_loss))
    print('Validation loss: {}/{:.6f}'.format(init_val_loss, val_losses_all[-1]))
    if summary_writer is not None:
        add_summary(summary_writer, init_val_loss, 0, 'val')

try:
    for epoch in range(start_epoch, args.epochs):
        start_t = time.time()
        network.train()
        train_outs, train_targets = [], []
        for batch_id, (feat, targ, _, _) in enumerate(train_loader):
            if (args.arch == 'IMUNet_'):
                feat = feat.unsqueeze(1)
            feat, targ = feat.to(device), targ.to(device)
            optimizer.zero_grad()
            pred = network(feat)
            train_outs.append(pred.cpu().detach().numpy())
            train_targets.append(targ.cpu().detach().numpy())
            loss = criterion(pred, targ)
            loss = torch.mean(loss)
            loss.backward()
            optimizer.step()
            step += 1
        train_outs = np.concatenate(train_outs, axis=0)
        train_targets = np.concatenate(train_targets, axis=0)
        train_losses = np.average((train_outs - train_targets) ** 2, axis=0)

        end_t = time.time()
        print('-------------------------')
        print('Epoch {}, time usage: {:.3f}s, average loss: {}/{:.6f}'.format(
            epoch, end_t - start_t, train_losses, np.average(train_losses)))
        train_losses_all.append(np.average(train_losses))

        if summary_writer is not None:
            add_summary(summary_writer, train_losses, epoch + 1, 'train')
            summary_writer.add_scalar('optimizer/lr', optimizer.param_groups[0]['lr'], epoch)

        if val_loader is not None:
            network.eval()
            val_outs, val_targets = run_test(network, val_loader, device)
            val_losses = np.average((val_outs - val_targets) ** 2, axis=0)
            avg_loss = np.average(val_losses)
            print('Validation loss: {}/{:.6f}'.format(val_losses, avg_loss))
            scheduler.step(avg_loss)
            if summary_writer is not None:
                add_summary(summary_writer, val_losses, epoch + 1, 'val')
            val_losses_all.append(avg_loss)
            if avg_loss < best_val_loss:
                best_val_loss = avg_loss
                if args.out_dir and osp.isdir(args.out_dir):
                    model_path = osp.join(args.out_dir, 'checkpoints', 'checkpoint_best.pt')
                    torch.save({'model_state_dict': network.state_dict(),
                                'epoch': epoch,
                                'optimizer_state_dict': optimizer.state_dict()}, model_path)
                    print('Model saved to ', model_path)

        total_epoch = epoch

except KeyboardInterrupt:
    print('-' * 60)
    print('Early terminate')

print('Training complete')
if args.out_dir:
    model_path = osp.join(args.out_dir, 'checkpoints', 'checkpoint_latest.pt')
    torch.save({'model_state_dict': network.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'epoch': total_epoch}, model_path)
    print('Checkpoint saved to ', model_path)

    # return train_losses_all, val_losses_all

NameError: name 'time' is not defined

TypeError: train() missing 1 required positional argument: 'args'